In [1]:
import json
import os
import sys
import time
from itertools import groupby
from typing import Any, List, Mapping, Optional, Union

import requests

In [2]:
perror = lambda *a, **kw: print(*a, **kw, file=sys.stderr)


def get(url: str, *keys: List[str]) -> Mapping[str, Any]:
    '''
    Raises:
      - requests.JSONDecodeError if resp.ok, but content is unparseable.
      - requests.HTTPError if not resp.ok.
    '''
    # Prepare headers.
    headers = {'accept': 'application/json'}
    apikey = os.environ.get('BEACONCHAIN_APIKEY')
    if apikey:
        headers['apikey'] = apikey
    resp = requests.get(url, headers=headers)
    if resp.ok:
        ans = resp.json()
        for k in keys:
            ans = ans[k]
        return ans
    perror('Status code :', resp.status_code)
    perror('Reason      :', resp.reason)
    perror('Text        :', resp.text)
    raise requests.HTTPError(resp.reason)

In [3]:
def history(pubkey: str, epoch: Optional[int] = None) -> Mapping[str, Any]:
    epoch = '' if epoch is None else f'latest_epoch={epoch}&'
    url = f'https://beaconcha.in/api/v1/validator/{pubkey}/balancehistory?{epoch}limit=1'
    return get(url, 'data')

In [4]:
def index(pubkey: str) -> int:
    url = f'https://beaconcha.in/api/v1/validator/{pubkey}'
    return get(url, 'data', 'validatorindex')

In [5]:
def performance(pubkey: str) -> Mapping[str, Any]:
    url = f'https://beaconcha.in/api/v1/validator/{pubkey}/performance'
    return get(url, 'data')

In [6]:
EEK = 2048
STEP = EEK

def fetch(validators):
    pubkeys = list(validators.keys())
    frames = {}
    for epoch in range(STEP, 193000+1, STEP):
        frames[epoch] = {}
        for key, group in groupby(pubkeys, key=lambda k: pubkeys.index(k) // 64):
            vs = ','.join(group)
            time.sleep(6)  # The free plan is limited to 10 requests per minute.
            data = history(vs, epoch)
            for v in data:
                index = v['validatorindex']
                frames[epoch][index] = v
    return frames

In [7]:
%%time 

with open('validators.json', 'r', encoding='utf-8') as f:
    validators = json.load(f)
frames = fetch(validators)

CPU times: user 13.3 s, sys: 497 ms, total: 13.8 s
Wall time: 1h 1min 17s


In [12]:
with open('frames.json', 'w', encoding='utf-8') as f:
    json.dump(frames, f)